In [1]:
from langchain import hub
prompt = hub.pull("tyfann/llm4commit-zeroshot")

In [2]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-0rLvuRkMiD4Mw25QYygh6rUlZVjpQWNGNF4yez7z3PZ7yCOm",
    # api_key="sk-tMbkq3K1iO5vf0FRMlrmzslGXJZwE0us3mve4QXuvpnZcumG",
    base_url="https://api.chatanywhere.tech/v1"
    # base_url="https://api.chatanywhere.cn/v1"
)

In [3]:
def gpt_35_api(messages: list):

    completion = client.chat.completions.create(model="gpt-3.5-turbo-0125", messages=messages, temperature=0.5)
    return completion.choices[0].message.content

In [4]:
import json
from tqdm import tqdm

In [5]:
with open('../data/chronicle/chronicle_data.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

In [6]:
org_data[:5]

[{'msg': 'Fixed loading terms from some GR pages that had a space after the class type',
  'diff': 'diff --git a/src/DataSources/Goodreads.cs b/src/DataSources/Goodreads.cs @@ -520,7 +520,8 @@ private XRay.Term GetTerm(string baseUrl, string relativeUrl)\nresult.DescUrl = tempUri.ToString();\nHtmlDocument charDoc = new HtmlDocument();\ncharDoc.LoadHtml(HttpDownloader.GetPageHtml(tempUri.ToString()));\n- HtmlNode mainNode = charDoc.DocumentNode.SelectSingleNode("//div[@class=\'mainContentFloat\']");\n+ HtmlNode mainNode = charDoc.DocumentNode.SelectSingleNode("//div[@class=\'mainContentFloat\']")\n+ ?? charDoc.DocumentNode.SelectSingleNode("//div[@class=\'mainContentFloat \']");\nresult.TermName = mainNode.SelectSingleNode("./h1").InnerText;\nmainNode = mainNode.SelectSingleNode("//div[@class=\'grey500BoxContent\']");\nHtmlNodeCollection tempNodes = mainNode.SelectNodes("//div[@class=\'floatingBox\']");\n',
  'lang': 'C#'},
 {'msg': 'Fixed labels on characters/terms in BaseDB',
  'diff'

In [7]:
import random
random.shuffle(org_data)
org_data[:5]

[{'msg': 'fix last typos',
  'diff': 'diff --git a/sc2/bot_ai.py b/sc2/bot_ai.py @@ -346,7 +346,7 @@ class BotAI:\nif len(worker_pool) > len(deficit_mining_places):\nall_minerals_near_base = [\nmineral\n- for mineral in self.mineral_fields\n+ for mineral in self.state.mineral_field\nif any(mineral.distance_to(base) <= 8 for base in self.townhalls.ready)\n]\n# distribute every worker in the pool\n@@ -376,7 +376,7 @@ class BotAI:\nmineral for mineral in self.state.mineral_field if mineral.distance_to(current_place) <= 8\n]\ntarget_mineral = max(local_minerals, key=lambda mineral: mineral.mineral_contents)\n- self.actions.append(worker.gather(target_mineral))\n+ actions.append(worker.gather(target_mineral))\n# more workers to distribute than free mining spots\n# send to closest if worker is doing nothing\nelif worker.is_idle and all_minerals_near_base:\n',
  'lang': 'Python'},
 {'msg': 'Add null check for compiledConstructor',
  'diff': 'diff --git a/src/NLog/Config/ServiceRepositoryInter

In [8]:
gpt_msg = []
for data in tqdm(org_data[:100], total=len(org_data[:100]), desc="Processing documents"):
    messages = prompt.invoke(
        {"DIFF": data['diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    gpt_msg.append(gpt_35_api(example_prompt))

for item, msg in zip(org_data[:100], gpt_msg):
    item['chatgpt'] = msg

Processing documents: 100%|██████████| 100/100 [01:58<00:00,  1.19s/it]


In [10]:
import os
output_file = '../data/chronicle/chronicle_chatgpt.json'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data[:100], f, ensure_ascii=False, indent=4)


## COT

In [11]:
prompt = hub.pull("tyfann/llm4commit-cot")
gpt_cot_msg = []

example_q = """"diff --git a/java/client/src/org/openqa/selenium/htmlunit/HtmlUnitAlert.java  b/java/client/src/org/openqa/selenium/htmlunit/HtmlUnitAlert.java \npublic void sendKeys(String keysToSend){ \npublic void authenticateUsing(Credentials credentials){}+@Override+public void setCredentials(Credentials credentials){+}++@Override \npublic void handleAlert(Page page,String message){ \nQueue<String>queue=queues.get(page); \n" What is the commit message of this code diff?"""
example_a = """First, the scope of code changes is to add a method, and second, according to the filename "HtmlUnitAlert", and the function of the file could be an alert, the modification "@Overide" is an annotation which can solve the compilation issue, so the modification can help solve the compilation problem to the alert file. So the commit message for the code diff is "fix compilation issue because of new method in Alert"."""
for data in tqdm(org_data[31:100], total=len(org_data[31:100]), desc="Processing documents"):
    messages = prompt.invoke(
        {"Example_Question": example_q, "Example_Answer": example_a, "Question": data['diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    gpt_cot_msg.append(gpt_35_api(example_prompt))

Processing documents: 100%|██████████| 69/69 [01:47<00:00,  1.56s/it]


In [12]:
len(gpt_msg), len(gpt_cot_msg)

(100, 100)

In [13]:
import re

def preprocess(text):
    if re.search(r'commit\s*message', text, re.IGNORECASE):
        pattern = r'\"([^"]+)'
        match = re.search(pattern, text)
        if match:
            return match.group(1)
        else:
            return text
    else:
        return text

In [14]:
for item, msg, cot_msg in zip(org_data[:100], gpt_msg, gpt_cot_msg):
    item['chatgpt_cot'] = preprocess(cot_msg)
    item['chatgpt_v2'] = msg

output_file = '../../bleu/all_result_cot.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data[:100], f, ensure_ascii=False, indent=4)

In [9]:
for item, msg in zip(org_data[:100], gpt_msg):
    item['chatgpt_v3'] = msg

output_file = '../../bleu/all_result_chatgpt.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data[:100], f, ensure_ascii=False, indent=4)